# Interações entre proteínas e entre domínios
---

**Autor:** Lucas Machado Moschen

**Professor:** Luciano Guimarães de Castro 

## Resumo
---

Um estudo sobre como redes de interação proteína-proteína podem inferir interações domínio-domínio utilizando uma abordagem de programação linear e inteira. 

## Sumário
---

1. [Introdução](#intro)
2. [O problema de interação domínio-domínio](#ddip-problem)
3. [Formulação com programação linear](#linear-program)
4. [Experimentação com dados hipotéticos](#experiments)
    - [Visualização dos dados hipotéticos](#hipothetical)
    - [Construindo o conjunto $\mathcal{D}$ e do grafo $\mathcal{B}$](#build-D-B)
    - [Programa Linear e Inteiro](#pli)
6.  [Adicionando mais realidade](#reality)
6. [Experimentação com dados reais](#real-data)
    - [Proteínas e seus domínios](#protein-domain)
    - [Rede PPI](#rede-ppi)
    - [Grafo bipartido $\mathcal{B}$](#bipartido)
    - [Modelo final](#final-model)
7. [Conclusão](#conclusion)

[Referências](#references)


## Introdução <a id="intro"></a>
--- 

<p style = "text-align: justify"> 
    Proteínas são <i>macromoléculas</i> formadas por cadeias de aminoácidos. Elas são construídas em formato <i>modular</i>, compostas por <i>domínios</i>, que são unidades da proteína funcionais ou estruturais independentes do resto. Muitas são construídas por dois ou mais domínios dentro de um repertório relativamente pequeno. O que variam são as combinações e a ordem de disposição. Frequentemente, os domínios individuais têm funções específicas, como catalisar uma reação ou ligação de uma molécula particular [1,2].
</p>

<center>
    <figure>
      <img src="images/figure-protein-domains.png" alt="protein-domains" style="width:50%">
        <figcaption><i><b>Fig.1</b> Representação da proteína citoplasmática Nck com seus três domínios SH3 e outro domínio SH2. Cada um desses domínios possui  um parceiro de ligação em outras proteínas. Disponível em <a href="https://www.ebi.ac.uk/training/online/courses/protein-classification-intro-ebi-resources/protein-classification/what-are-protein-domains/"> EMBL-EBI </a>. </i></figcaption>
    </figure>
</center>

<p style = "text-align: justify"> 
Proteínas interagem umas com as outras nas células e o conjunto dessas interações é chamado de <i>Rede de Interações Proteína-Proteína (Rede PPI)</i>. Matematicamente, representamo elas como um grafo (direcionado ou não direcionado) cujos nós são as proteínas e cujas arestas são as interações entre as proteínas, verificadas experimentalmente. Uma rede PPI é a base molecular do desenvolvimento e do funcionamento dos organismos. Seu entendimento é essencial, tanto para a compreensão de estados normais e doentes de células, quanto para o desenvolvimento de drogas. Ela pode ser construída para descrever uma série de informações, como, por exemplo, proteínas que estão na mesma regiões ao mesmo tempo; proteínas que são correguladas; proteínas que estão envolvidas com alguma função biológica; entre outras [2]. Em particular, o tópico de minha possível dissertação vai nesse sentido e, por esse motivo, foi o tema escolhido. 
</p>

<center>
    <figure>
        <img src="images/ppi-network.png" alt="ppi-network-example" style="width:50%"> 
        <figcaption><i><b>Fig. 2</b> Exemplo de rede PPI usando NetworkAnalysist. <a href="https://www.researchgate.net/figure/An-overview-of-the-PPI-network-The-PPI-network-was-generated-using-NetworkAnalyst-Red_fig5_323136513"> Desenvolvido por Wei Zhong, et al. </i></a></figcaption>
    </figure>
</center>

<p style = "text-align: justify"> 
Como as interações entre proteínas geralmente ocorrem via domínios ao invés de toda a molécula, entender como os domínios interagem entre si pode facilitar a obtenção de redes PPI mais completas, com menor custo e tempo. Por esse motivo, predizer interações entre domínios (DDI) é um passo importante para a predição de PPI. Chamamos esse problema de <i>problema de interação domínio-domínio (DDIP)</i>. 
</p>

## O problema de interação domínio-domínio (DDIP) <a id="ddip-problem"></a>
---

<p style="text-align:justify">
    Denotamos uma rede PPI conhecida por um grafo não direcionado $\mathcal{N} = (\mathcal{P}, \mathcal{E})$, em que $\mathcal{P}$ é o conjunto de proteínas e $\mathcal{E}$ é o conjunto de pares de proteínas que interagem, dado algum experimento. Para cada proteína $P \in \mathcal{P}$, supomos conhecidos seus domínios $D_P$ e definimos $\mathcal{D} = \{\{d_1, d_2\} : d_1 \in D_{P_1}, d_2 \in D_{P_2}, \text{ e } \{P_1, P_2\} \in \mathcal{E}\}$, isto é, o conjunto dos pares (não ordernados) de <b> possíveis </b> interações domínio domínio. Então, queremos inferir, para cada $I = \{P_1, P_2\} \in \mathcal{E}$, quais pares de domínios distintos em $D_{P_1}$ e $D_{P_2}$ explicam a interação entre as proteínas [1].
</p>

<p style="text-align:justify">
    Vamos assumir que as interações domínio-domínio são <i>conservadas</i> entre as várias interações entre proteínas, isto é, elas tendem a repetir em diversos contextos. Também supomos que a interação entre proteínas evolui de forma parcimoniosa e que o conjunto DDI é bem aproximado pelo menor conjunto de interações necessárias para explicar a rede PPI, ou seja, sabemos que as relações domínio-domínio influenciam fortemente as relações proteína-proteína e utilizamos um método baseado em parcimônia para explicar essa inferência. O objetivo será, portanto, minimizar o número de interações domínio-domínio que justifiquem a interação entre proteínas observada na rede [3].
</p>


<p style="text-align:justify"> 
    Seja $\mathcal{B}$ um grafo bipartido não direcionado com dois conjuntos independentes: $\mathcal{P}^2 = \mathcal{E}$ (conjunto dos pares de interações proteína-proteína) e $\mathcal{D}$ (pares de domínios), em que um elemento $\{P_1, P_2\} \in \mathcal{P}^2$ é ligado a $\{d_1, d_2\} \in \mathcal{D}$ se $d_1 \in D_{P_1}$ e $d_2 \in D_{P_2}$.  
</p>

<figure>
    <center>
        <img src="images/ddi-ppi-example.svg" alt="test" style="width:50%" >    
        <figcaption><i><b> Fig. 3 </b> Exemplo simplificado de rede PPI com respectivos domínios e a construção do grafo bipartido correspondente.</i></figcaption>
    </center>
<\figure>

<p style="text-align:justify">
    <b>Definição (Cobertura):</b> Um conjunto $S \subset \mathcal{D}$ é <i>cobertura DDI</i> se cada nó em $\mathcal{P}^2$ é adjacente a pelo menos um nó de $S$. Trivialmente $\mathcal{D}$ é uma cobertura por definição, portanto, sabemos que ela existe.  
</p>

<p style="text-align:justify">
Para cada nó $x \in \mathcal{P}^2$, denotamos por $\mathcal{V}(x)$ o conjunto de nós em $\mathcal{D}$ que são vizinhos de $x$. Podemos resumir o problema, portanto, com a seguinte a proposição:
</p>

> Dado um grafo bipartido $\mathcal{B}$ derivado de $\mathcal{P}^2$ e $\mathcal{D}$, encontre uma cobertura DDI de tamanho mínimo em $\mathcal{B}$.

Nesse caso, o conjunto solução $S^*$ é o conjunto mínimo que aproxima a explicação da interação entre proteínas. 

## Formulação com programação linear <a id="linear-program"></a>
---

Vamos definir as variáveis binárias $x_{ij}$ que indicam se o nó $\{i,j\} \in \mathcal{D}$ pertence à cobertura $S$. 

Queremos minimizar o tamanho dessa cobertura, portanto, a função objetivo a ser minimizada é 

$$\sum_{\{i,j\} \in \mathcal{D}} x_{ij}.$$

Para cada nó $v \in \mathcal{P}^2$, ou seja, para cada interação entre duas proteínas, queremos que pelo uma dupla de seus domínios interajam entre si. Assim, pelo menos um par de domínios vizinhos de $v$ deve existir para explicar a PPI. Definimos a restrição, para cada $v$, 

$$\sum_{\{i,j\} \in \mathcal{V}(v)} x_{ij} \ge 1$$

Além disso, temos, é claro, $0 \le x_{ij} \le 1, x_{ij} \in \mathbb{Z}$. Sabemos que espaço viável é não vazio, pois colocando todas as variáveis em 1, todas as restrições são verdadeiras. 

## Experimentação com dados hipotéticos <a id="experiments"></a>
---

Nessa seção, vamos usar o ferramental de otimização na linguagem de programação *Julia* para fazer experimentos do problema. Vamos usar um dataset hipotético desenvolvido em [4]. 

In [1]:
using Pkg
#pkg"add JuMP GLPK CSV DataFrames Cbc"
pkg"activate ../."
pkg"instantiate"
using JuMP, GLPK, CSV, LinearAlgebra, DataFrames, Cbc

  Activating environment at `~/Documents/linear-programming/Project.toml`


### Visualização do dado hipotético <a id="hipothetical"></a>

Podemos visualizar os dados e trnascrever para uma estrutura de dicionário. 

In [2]:
io = open("data/hypothetical_network.txt", "r")
print(read(io, String))

#######
# NODES
#######
0	P0	0	NULL	NULL	NULL	Hypothetical protein	Yeast	NULL
1	P1	0	NULL	NULL	NULL	Hypothetical protein	Yeast	NULL
2	P2	0	NULL	NULL	NULL	Hypothetical protein	Yeast	NULL
3	P3	0	NULL	NULL	NULL	Hypothetical protein	Yeast	NULL
4	P4	0	NULL	NULL	NULL	Hypothetical protein	Yeast	NULL
5	P5	0	NULL	NULL	NULL	Hypothetical protein	Worm	NULL
6	P6	0	NULL	NULL	NULL	Hypothetical protein	Worm	NULL
7	P7	0	NULL	NULL	NULL	Hypothetical protein	Worm	NULL
8	P8	0	NULL	NULL	NULL	Hypothetical protein	Worm	NULL
9	P9	0	NULL	NULL	NULL	Hypothetical protein	Human	NULL
10	P10	0	NULL	NULL	NULL	Hypothetical protein	Human	NULL
11	P11	0	NULL	NULL	NULL	Hypothetical protein	Human	NULL
#######
# EDGES
#######
0	0	1	h
1	0	2	h
2	0	3	h
3	0	4	h
4	1	2	h
5	1	3	h
6	1	4	h
7	5	6	h
8	5	7	h
9	6	8	h
10	7	8	h
11	9	10	h
12	10	11	h
#########################
# NODE -> DOMAIN MAPPINGS
#########################
0	Yellow
0	Blue
1	Azure
1	Orange
1	Green
1	Red
1	Blue
1	Red
2	Red
3	Red
3	Orange
4	Violet
4	Green
4	Red
4	Blue
4	Red

In [3]:
protein_domain = Dict("P₁"=>["Yellow", "Blue"], 
                      "P₂"=>["Azure", "Orange", "Green", "Red", "Blue", "Red"], 
                      "P₃"=>["Green"], 
                      "P₄"=>["Red", "Orange"] , 
                      "P₅"=>["Violet", "Green", "Red", "Blue", "Red", "Yellow", "Red"] , 
                      "P₆"=>["Orange","Red"] , 
                      "P₇"=>["Blue"] , 
                      "P₈"=>["Blue"] , 
                      "P₉"=>["Red", "Green"] , 
                      "P₁₀"=>["Red"] , 
                      "P₁₁"=>["Blue", "Orange", "Violet", "Azure", "Green"], 
                      "P₁₂"=>["Red"]);

ppi_network = [
    ("P₁", "P₂"), ("P₁", "P₃"), ("P₁", "P₄"), ("P₁", "P₅"),
    ("P₂", "P₃"), ("P₂", "P₄"), ("P₂", "P₅"), 
    ("P₆", "P₇"), ("P₆", "P₈"), 
    ("P₇", "P₉"), ("P₈", "P₉"), ("P₁₀", "P₁₁"), ("P₁₁", "P₁₂")
];

B_P = Dict([
    ("P₁", "P₂")=>[], ("P₁", "P₃")=>[], ("P₁", "P₄")=>[], ("P₁", "P₅")=>[],
    ("P₂", "P₃")=>[], ("P₂", "P₄")=>[], ("P₂", "P₅")=>[], 
    ("P₆", "P₇")=>[], ("P₆", "P₈")=>[], 
    ("P₇", "P₉")=>[], ("P₈", "P₉")=>[], ("P₁₀", "P₁₁")=>[], ("P₁₁", "P₁₂")=>[]
]);

### Construindo o conjunto $\mathcal{D}$ e do grafo $\mathcal{B}$ <a id="build-D-B">

Com a rede PPI definida, podemos contruir o conjunto $\mathcal{D}$ com as relações entre os domínios. Com isso, teremos o grafo bipartido $\mathcal{B}$. Vamos separar o grafo apenas com as informações que precisamos: $B_P$ indica, para cada interação PP, os seus vizinhos em $\mathcal{B}$ e $B_D$ indica, para cada interação DD, a quantidade de vizinhos em $\mathcal{B}$. 

In [4]:
D = Dict()
for i in ppi_network
    for d1 in protein_domain[i[1]]
        for d2 in protein_domain[i[2]]
            if d1 != d2
                D[Set([d1, d2])] = get(D, Set([d1, d2]), length(D)+1)
                B_P[i] = union(B_P[i], D[Set([d1, d2])])
            end
        end
    end
end
B_D = zeros(length(D))
for v in B_P
    B_D[v[2]] .+= 1
end

### Programa Linear e Inteiro <a id="pli">

Vamos, enfim, contruir o programa. Vamos chamar $ij = u$. Denote que os vizinhos de cada interação entre proteínas é dado por $B[i]$. 

In [5]:
ddip_model = Model(GLPK.Optimizer);

@variable(ddip_model,x[u=1:length(D)], Bin);

@objective(ddip_model, Min, sum(x));

@constraint(ddip_model, [sum(x[B_P[i]]) for i in ppi_network] .>= 1);

Com o problema definido, podemos otimizar! Felizmente, o algoritmo terminou tudo bem! Ele encontrou duas variáveis iguais a 1 (como podemos ver abaixo), sendo que uma delas satisfaz quase todas as restrições (exceto as que envolvem $P_3$). 

In [6]:
optimize!(ddip_model)

termination_status(ddip_model)

OPTIMAL::TerminationStatusCode = 1

In [7]:
objective_value(ddip_model)

2.0

Podemos observar que as variáveis 8 (interação entre Green e Blue) e 9 (interação entre Red e Blue) foram as únicas que marcadas.

In [8]:
value.(x)[[8,9]]

2-element Vector{Float64}:
 1.0
 1.0

## Adicionando mais realidade <a id="reality"></a>
---

<p style="text-align:justify">
Obtemos um resultado que diz quais interações domínio-domínio são necessárias para explicar as interações entre proteínas que observamos com um método concebido pela ideia de parcimônia. Nesse caso, obtemos um resultado 0 ou 1. Podemos, todavia, indicar a probabilidade de uma interação domínio a domínio existir. Observe que a restrição fica inalterada. Como observamos uma interação proteína-proteina $(P_1, P_2) \in \mathcal{P}^2$, podemos dizer que
</p>
    
$$
1 = \mathbb{P}\left((P_1, P_2) \in \mathcal{P}^2\right) = \mathbb{P}\left((d_1, d_2) \in \mathcal{D}, \text{ para alguns } d_1 \in D_{P_1}, d_2 \in D_{P_2}\right) \le \sum_{d_1 \in D_{P_1}, d_2 \in D_{P_2}} \mathbb{P}\left((d_1, d_2) \in \mathcal{D}\right)
$$

<p style="text-align:justify">
Suponha agora, que existem duas soluções de mesmo tamanho. Quando isso acontece, vamos dar um peso maior para aquela com maior sentido biológico, isto é, se um no $(d_1, d_2) \in \mathcal{D}$ tem mais vizinhos do que $(d_3, d_4) \in \mathcal{D}$, queremos que ele tenha mais chance de estar no conjunto ótimo. Isto é, vamos adicionar pesos que beneficiam nós com mais vizinhos! Assim, a função objetivo fica 
</p>

$$\sum_{\{i,j\} \in \mathcal{D}} w_{ij}x_{ij}.$$

em que $w_{ij} = 1/\mathcal{V}(ij)$. No nosso exemplo, a resposta ficará inalterada, mas é interessante ter um critério de desempate biológico. 

**Observação:** Colocar a variável entre 0 e 1, mas não binária, também simplifica as contas quando tivermos mais variáveis no [problema real](#real-data).

In [9]:
ddip_model2 = Model(GLPK.Optimizer);

@variable(ddip_model2, 1 >= x[u=1:length(D)] >= 0.0);

@objective(ddip_model2, Min, sum(x./B_D));

@constraint(ddip_model2, [sum(x[B_P[i]]) for i in ppi_network] .>= 1);


optimize!(ddip_model2)
objective_value(ddip_model2)

0.23376623376623376

In [10]:
print(value.(x))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

<p style="text-align: justify">
As redes de PPI não são sempre interamente corretas. Elas são sujeitas ao ruído na experimentação. Esse ruído pode ser lidado de formas diferentes: adicionar pesos às interações PP indicando a probabilidade de elas serem verdadeiras, ou remover algumas restrições de forma "aleatória". A primeira, no nosso caso, representaria pesos nos nós em $\mathcal{P}^2$ e é mais difícil de representar em um modelo determinístico. A segunda tem uma formulação interessante: permitimos a escolha de uma porcentagem $r$ de restrições, enquanto as outras são desconsideradas, isso é, consideraremos apenas uma porcentagem $r$ das interações PP. Vamos fazer isso do seguinte modo: 
    
Defina $y_v$ uma variável binária para cada restrição. Queremos que 
</p>

$$\sum_{\{i,j\} \in \mathcal{V}(v)} x_{ij} \ge y_v, \text{ para cada } v \in \mathcal{P}^2$$

E, além disso, restringimos

$$
\sum_{v \in \mathcal{P}^2} y_v \ge r |\mathcal{P}^2| 
$$

Essa restrição pode fazer tirar uma interação PP quando ela tem pouca influência ou advém de um ruído. 

In [11]:
ddip_model3 = function(r)

    model = Model(GLPK.Optimizer);

    @variable(model, 1 >= x[u=1:length(D)] >= 0.0);
    @variable(model, y[v=1:length(B_P)], Bin);

    @objective(model, Min, sum(x./B_D));

    @constraint(model, [sum(x[B_P[i]]) for i in ppi_network] .>= y);
    @constraint(model, sum(y) >= r*length(B_P));

    optimize!(model)
    return objective_value(model), value.(x), value.(y)
    end;

Vamos experimentar alguns valores de $r$

In [12]:
obj1, xx1, yy1 = ddip_model3(0);
obj2, xx2, yy2 = ddip_model3(0.5);
obj3, xx3, yy3 = ddip_model3(0.9);

Quando $r = 0$, permitimos que todas as restrições sejam deixadas de lado. Certamente, estamos colocando ruído a mais. Com $r = 0.5$, para satisfazer metade das restrições, apenas um nó é suficiente. Para $r = 0.9$, apesar de ter considerado um pouco de ruído, ainda mantivemos a mesma escolha de quando $r =1$. 

In [13]:
(obj1, obj2, obj3)

(0.0, 0.09090909090909091, 0.23376623376623376)

In [14]:
print(xx2)
print("\n")
print(xx3)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

## Experimentação com dados reais <a id="real-data"></a>
---

Vamos utilizar essa modelagem em um problema real, com dados de [4]. 

In [15]:
pd_table = CSV.File("data/protein_domain.txt", delim='\t') |> DataFrame;
ppi_table = CSV.File("data/ppi_network.txt", delim='\t') |> DataFrame;
select!(ppi_table, Not(:"exp_class"));

O número de interações é 26.032, enquanto o número de mapas entre nós e domínios é 30.986. 

In [16]:
print(size(pd_table))
print(size(ppi_table))

(30986, 2)(26032, 3)

### Proteínas e seus domínios <a id="protein-domain"> 

Primeiro montamos o dicionário com cada proteína e seus respectivos domínios:

In [17]:
proteins = Dict()
for p in eachrow(pd_table)
    proteins[p[1]] = union(get(proteins, p[1], []), [p[2]])
end
proteins[100]

12-element Vector{Any}:
 "C2"
 "Pfam-B_11112"
 "Pfam-B_34975"
 "Pfam-B_44417"
 "Pfam-B_44419"
 "Pfam-B_5217"
 "Pfam-B_68720"
 "PH"
 "PI-PLC-X"
 "PI-PLC-Y"
 "SH2"
 "SH3_1"

### Rede PPI <a id="rede-ppi">

Montamos a rede PPI:

In [18]:
ppi_real = []
for i in eachrow(ppi_table)
    if i[2] != i[3]
        ppi_real = append!(ppi_real, [(i[2], i[3])])
    end
end

### Grafo bipartido $\mathcal{B}$ <a id="bipartido"> 

Montamos $\mathcal{B}$: 

In [19]:
B_p = Dict(i => [] for i in ppi_real);

In [20]:
D = Dict()
for i in ppi_real
    for d1 in proteins[i[1]]
        for d2 in proteins[i[2]]
            D[Set([d1, d2])] = get(D, Set([d1, d2]), length(D)+1)
            B_p[i] = union(B_p[i], D[Set([d1, d2])])
        end
    end
end
B_d = zeros(length(D))
for v in B_p
    B_d[v[2]] .+= 1
end

### Modelo Final <a id="final-model">

Agora podemos usar o modelo! Lembramos que ele tem muitas, mas muitas mais variáveis que nosso teste, então deve demorar bastante mais tempo para encerrar o programa, pelo menos na primeira rodagem! Observe que o número de variáveis é de quase 200mil.  

In [23]:
length(D) + length(B_p)

198935

In [21]:
ddip_model_real = function(r, D, BP, BD, PPI)

    model = Model(Cbc.Optimizer);

    @variable(model, 1.0 >= x[u=1:length(D)] >= 0.0);
    @variable(model, y[v=1:length(B_P)], Bin);

    @objective(model, Min, sum(x./BD));

    @constraint(model, [sum(x[BP[i]]) for i in PPI] .>= 1);
    @constraint(model, sum(y) >= r*length(B_P));

    optimize!(model)
    return objective_value(model), value.(x), value.(y)
end;

In [22]:
@time obj_real1, xx_real1, yy_real1 = ddip_model_real(0.9, D, B_p, B_d, ppi_real);

167.731943 seconds (29.02 M allocations: 1.692 GiB, 0.46% gc time, 0.56% compilation time)
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 14023.8 - 0.18 seconds
Cgl0004I processed model has 6304 rows, 10233 columns (0 integer (0 of which binary)) and 19312 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 14023.8 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 

O programa teve sucesso e levou quase 3min para concluir o processo. Podemos ver qual a porcentagem (mínima) necessária para justificar as interações entre proteínas. Em torno de 9% por cento das interações domínio-domínio foram necessárias para justificar todo o aparato. 

In [24]:
sum(xx_real1)/length(xx_real1)

0.09398963730569948

Note que um pouco mais de 90% das restrições foram usadas, o que diminuí um pouquinho do possível ruído. 

In [25]:
sum(yy_real1)/length(yy_real1)

0.9230769230769231

## Conclusão <a id="conclusion"></a>
---

Podemos observar uma modelagem com otimização linear em um problema complexo de biologia. Apesar das diversas simplificações, o método simplex é simples de interpretar e é rápido para funcionar, o que, em muitos casos, é essencial para um conjunto de dados longos. No exemplo real, percebemos uma dificuldade no programa para encerrar a otimização, que levou quase 3min. 

## Referências <a id="references"></a>
---

[1] Althaus, Ernst & Klau, Gunnar & Kohlbacher, Oliver & Lenhof, Hans-Peter & Knut, Reinert. (2009). *Integer Linear Programming In Computational Biology*. J Proteome Res, Volume 5760 of Lecture Notes in Computer Science. 5760. 199-218. 10.1007/978-3-642-03456-5_14. 

[2] EMBL-EBI. Protein classification: An introduction to EMBL-EBI resources. Disponível [nesse site](https://www.ebi.ac.uk/training/online/courses/protein-classification-intro-ebi-resources/).

[3] Guimarães, K.S., Jothi, R., Zotenko, E. et al. Predicting domain-domain interactions using a parsimony approach. Genome Biol 7, R104 (2006). https://doi.org/10.1186/gb-2006-7-11-r104

[4] Riley, R., Lee, C., Sabatti, C. et al. Inferring protein domain interactions from databases of interacting proteins. Genome Biol 6, R89 (2005). https://doi.org/10.1186/gb-2005-6-10-r89
